# Serialized Population Demo

## Motivation
- Saves dtk state to file
- Loads file and continues, e.g. burn-in

## Configuration

## Created Files
- state-00001.dtk
- state-00010.dtk
- state-00050.dtk


## Load and Continue

## Manipulating a dtk File

- file access, writing, reading, compression
- https://github.com/InstituteforDiseaseModeling/DtkTrunk/tree/master/Scripts/serialization 


## Pycharm Demo
- Run eradication.exe and generate .dtk file at time step 1
- Open dtk file in Pycharm
- Show node content in debugger

## Demo
- Load dtk file at timestep 1
- Infect some indiviuals
- Save dtk file
- Start simulation with created dtk file
- https://github.com/tfischle-idmod/SerializedPopulation-Tools

In [1]:
import sys
sys.path.append("C:\\Users\\tfischle\\Github\\SerializedPopulation-Tools")
import change_serialized_population as csp
import pathlib
import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

In [2]:
serialized_file = "state-00001.dtk"
path = pathlib.PureWindowsPath(r"C:\Users\tfischle\Desktop\Eradication_2.21\output", serialized_file)

ser_pop = csp.SerializedPopulation(path)
node_0 = ser_pop.nodes[0]

In [3]:
# plot m_age 
ind_age = csp.getAttributeValues(node_0.individualHumans, "m_age")   # m_age C++ name

layout = go.Layout( title="m_age", 
                           bargap=0.2, bargroupgap=0.1, 
                           xaxis=dict(title="age in days"),
                           yaxis=dict(title="#individuals"))

data = [go.Histogram(x=ind_age)]
plotly.offline.iplot (
    { 'data': data, 
      'layout': layout}
)

In [4]:
# plot number of infections
fct = lambda ind: len(ind["infections"]) > 0
ind_values = csp.getAttributeValues(node_0.individualHumans, "m_age", filter_fct=fct)

data = [go.Histogram(x=ind_values)]
plotly.offline.iplot ({ 'data': data, 'layout': layout})

In [5]:
#print infection
import json
path_infection = pathlib.PureWindowsPath(r"C:\Users\tfischle\Desktop\Eradication_2.21\infection.json")
with open(path_infection, "r") as file:
    infection = json.load(file)
    print(json.dumps(infection, indent=4))

{
    "__class__": "Infection",
    "suid": {
        "id": 31
    },
    "duration": 7,
    "total_duration": 9.7328,
    "incubation_timer": 0,
    "infectious_timer": 9.7328,
    "infectiousness": 3.5,
    "infectiousnessByRoute": [],
    "StateChange": 0,
    "infection_strain": {
        "cladeID": 0,
        "geneticID": 0
    },
    "m_is_symptomatic": 1,
    "m_is_newly_symptomatic": 0
}


In [6]:
# load infection and append it to several individuals
ser_pop.addInfection(node_0,path_infection, lambda ind: ind["m_age"] > 30000)

In [7]:
# plot all individuals with an infection
fct = lambda ind: len(ind["infections"]) > 0
ind_values = csp.getAttributeValues(node_0.individualHumans, "m_age", filter_fct=fct)

data = [go.Histogram(x=ind_values)]
plotly.offline.iplot ({ 'data': data, 'layout': layout})

In [8]:
# Write to file
serialized_output_file = "dtk-added_infections.dtk"
output_path = pathlib.PureWindowsPath(r"C:\Users\tfischle\Desktop\Eradication_2.21", serialized_output_file)
ser_pop.close()
ser_pop.write(output_path)

## Result
- Simulation starts with 7 infected individuals

## Outlook / Future Plans / Ideas
- Distribute created population to several nodes
- Modularization i.e save/load parts of simulation, load parts from configuration ( e.g. laraval habitats)
- Create synthetic population
- Creation of individuals/infections? 
- Backwards compatibility, e.g. new/renamed variables?
- Testing